In this example, we will write out a NWChem input file based on a template

In [1]:
import streamm

In [2]:
import os 
from pprint import pprint
from pathlib2 import Path

First, we need to set the location of the templates directory cloned from http://github.com/NREL/streamm-tools

If you are running this example in the examples directory in the streamm-tools repo, the TEMPLATE_DIR should look like this

In [3]:
EXAMPLE_DIR = os.getcwd()

print(EXAMPLE_DIR)

TEMPLATE_DIR =  os.path.join(EXAMPLE_DIR,'..','templates','')

print(TEMPLATE_DIR)

/Users/tkemper/Development/streamm-tools/examples
/Users/tkemper/Development/streamm-tools/examples/../templates/


If not please set the `TEMPLATE_DIR` variable to the location of the templates

We will use the basic nwchem.nw example template 

In [4]:
temlate_file = 'nwchem.nw'

Create a NWChem calculation object

In [5]:
nwchem = streamm.NWChem('ethane_nw_sp')

Read in the ethane structure we creating in the buildingblocks example. 

.. Note::

    If you have not run the buildingblocks_example.ipynb example, please do so the create a `.xyz` file
    

In [6]:
need_files = ['ethane_struc.json']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run buildingblocks_example.ipynb")
        os.system("jupyter nbconvert --to python  buildingblocks_example.ipynb")
        os.system("python buildingblocks_example.py")

In [7]:
nwchem.strucC.tag = 'ethane'
nwchem.strucC.import_json()
print nwchem.strucC.print_properties()

 n_particles:8 
 n_bonds:7
 n_angles:12
 n_dihedrals:9
 n_impropers:0


Get the location of the template file

In [8]:
template_path =  os.path.join(TEMPLATE_DIR,temlate_file)

print template_path

/Users/tkemper/Development/streamm-tools/examples/../templates/nwchem.nw


Read in the template 

In [9]:
template_line = nwchem.read_lines(template_path)

In [10]:
print template_line

start test
 geometry GEOM units angstroms NOCENTER NOAUTOZ NOAUTOSYM
<coord>end


 BASIS 
 * LIBRARY <basis>
 end 
 SET geometry  GEOM 
 CHARGE  <charge>
 SCF 
 NOPEN 0
 <method> 
 SINGLET
 maxiter <maxiter> 
 end 
 TASK <task>




Set the properties dictionary to contain the information for our calculation

In [11]:
nwchem.properties['basis'] = '6-31g'
nwchem.properties['method'] = 'UHF'
nwchem.properties['charge'] = 0
nwchem.properties['spin_mult'] = 1
nwchem.properties['task'] = 'SCF '
nwchem.properties['coord'] = nwchem.strucC.write_coord()

Do a string replace of the dictionary keys to create an input string

In [12]:
input_str = nwchem.replace_keys(template_line,nwchem.properties)
print input_str

start test
 geometry GEOM units angstroms NOCENTER NOAUTOZ NOAUTOSYM
     C       1.34000000      -0.00000000       0.00000000 
     H       1.74000000      -0.00000000      -1.13137084 
     H       1.74000000       0.97979589       0.56568542 
     H       1.74000000      -0.97979589       0.56568542 
     C       0.00000000       0.00000000       0.00000000 
     H      -0.40000000       0.00000000       1.13137084 
     H      -0.40000000      -0.97979589      -0.56568542 
     H      -0.40000000       0.97979589      -0.56568542 
end


 BASIS 
 * LIBRARY 6-31g
 end 
 SET geometry  GEOM 
 CHARGE  0
 SCF 
 NOPEN 0
 UHF 
 SINGLET
 maxiter 100 
 end 
 TASK SCF 




In [13]:
file_name = '%s.nw'%(nwchem.tag)
with open(file_name,"w") as F:
    F.write(input_str)


Easy peasy!